<font size=6.5 color="blue">** Recommendations **</font>

# 寻找相近的用户

In [4]:
critics={
    'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,'Just My Luck': 3.0, 'Superman Returns': 3.5, 
                  'You, Me and Dupree': 2.5, 'The Night Listener': 3.0},
    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
                     'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 
    'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,'Superman Returns': 3.5, 'The Night Listener': 4.0},
    'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5},
    'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                     'You, Me and Dupree': 2.0}, 
    'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,'The Night Listener': 3.0, 'Superman Returns': 5.0, 
                      'You, Me and Dupree': 3.5},
    'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}} 

In [117]:
critics['Lisa Rose']['Lady in the Water']

2.5

## 相似度评价值
- 欧几里得距离（Euclidean Distance Score）
- 皮尔逊相关度（Pearson Correlation Score）

### 欧氏距离

**评价值：**
$$\frac{1}{1+欧氏距离}$$
结果在`[0,1]`之间，数值越大，越相关

In [1]:
from math import sqrt

In [6]:
def sim_distance(prefs,person1,person2):
    shared_items={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            shared_items[item]=1
        
    if len(shared_items)==0: return 0
    
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                        for item in shared_items if item in prefs[person2]])
    
    return 1/(1+ sqrt(sum_of_squares))      #为保持和书本“一样错”，正确语句：1/(1+ sqrt(sum_of_squares))

In [7]:
sim_distance(critics,'Lisa Rose', 'Gene Seymour')

0.29429805508554946

**两两计算欧几里得距离：**

In [121]:
def calcu(prefs,func=sim_distance):
    key_l = list(critics.keys())    
    score_d={}
    for i in range(0, len(key_l)-1):      #列表index是从“0”开始的
        for m in range(i+1, len(key_l)):
            score = func(critics,key_l[i],key_l[m])
            score_d[key_l[i]+' : '+ key_l[m]]=score      
    return score_d

In [122]:
score_d=calcu(critics)

In [123]:
score_sorted= sorted(score_d.items(),key=lambda item:item[1])  #对字典进行排序，返回d.items()
# keys_sorted_by_value = sorted(score_d,key=lambda x:score_d[x])    #按键值排序，返回键的顺序
score_sorted[-1]

('Gene Seymour : Jack Matthews', 0.8)

### 皮尔逊相关系数
参考：
1. [如何理解皮尔逊相关系数（Pearson Correlation Coefficient-知乎）？](https://www.zhihu.com/question/19734616)
2. [皮尔逊积矩相关系数-WikiPedia](https://zh.wikipedia.org/wiki/%E7%9A%AE%E5%B0%94%E9%80%8A%E7%A7%AF%E7%9F%A9%E7%9B%B8%E5%85%B3%E7%B3%BB%E6%95%B0)
$$r=\frac{\sum (x - \overline{x}) (y - \overline{y}) }{\sqrt{\sum (x - \overline{x})^2 (y - \overline{y})^2}}$$
<font color='red'>分母为0的情况如何剔除？</font>

In [124]:
from scipy.stats import pearsonr  #return r and p-value

In [204]:
def sim_pearson(prefs,p1,p2):
    shared_items={}
    for item in prefs[p1]:
        if item in prefs[p2]:
            shared_items[item]=1
    n=len(shared_items)
    if n<=0: return 0
    
#     x1=[prefs[p1][item] for item in shared_items]
#     x2=[prefs[p2][item] for item in shared_items]
#     r,p_value= pearsonr(x1,x2)
#     if p_value==1: return 0
    
    sum1=sum([prefs[p1][it] for it in shared_items])
    sum2=sum([prefs[p2][it] for it in shared_items])
  
    # Sums of the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in shared_items])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in shared_items])
  
    # Sum of the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in shared_items])
  
    # Calculate r (Pearson score)
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den

    return r

In [197]:
sim_pearson(critics,'Lisa Rose','Toby')

0.99124070716193036

In [129]:
score_r=calcu(critics,sim_pearson)

In [130]:
score_r_sorted= sorted(score_r.items(),key=lambda item:item[1])
score_r_sorted[-1]

('Michael Phillips : Claudia Puig', 1.0)

### 其他函数
- [Jaccard系数-百度百科](https://baike.baidu.com/item/Jaccard%E7%B3%BB%E6%95%B0/6784913?fr=aladdin)
- [曼哈顿距离-百度百科](https://baike.baidu.com/item/%E6%9B%BC%E5%93%88%E9%A1%BF%E8%B7%9D%E7%A6%BB/743092?fr=aladdin)
- Tanimoto分值

## 打分计算

In [131]:
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]    #字典转化成 元组 列表，便于排序
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [132]:
topMatches(critics,'Toby',n=2)

[(0.9912407071619299, 'Lisa Rose'), (0.9244734516419049, 'Mick LaSalle')]

# 推荐物品

In [205]:
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals = {}
    simSums= {}
    for other in prefs:
        if other == person: continue    #跳出本次循环
        sim =similarity(prefs,person,other)
        if sim<=0: continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item]==0:
                totals.setdefault(item,0)
                totals[item]+=sim*prefs[other][item]
                simSums.setdefault(item,0)
                simSums[item]+=sim
    rankings=[(total/simSums[item], item) for item,total in totals.items()]
    
    rankings.sort()
    rankings.reverse()
    return rankings

In [147]:
getRecommendations(critics,'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

# 匹配商品
相近商品匹配

In [148]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            result[item][person]=prefs[person][item]
    return result

In [149]:
movies = transformPrefs(critics)
topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [150]:
getRecommendations(movies, 'Just My Luck')    #对未观看该电影的人评分进行预测

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

# 链接推荐系统
基于[del.icio.us](www.del.icio.us)
<br>网站服务器切换，API错误，待完成。
<br>日期：2018/7/E
<br>Page：19

# 基于物品的推荐

In [183]:
def calculateSimilarItems(prefs,n=10,p=False):
    result={}
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        c+=1
        if p and c%100==0 : print("%d / %d" %(c,len(itemPrefs))) 
        scores =topMatches(itemPrefs, item, n=n, similarity=sim_distance)
        result[item] =scores
    return result

构件物品相似度数据集 `itemsim`
<br>元组组成的列表，相当于一个字典，比字典更“稳定”

In [152]:
itemsim=calculateSimilarItems(critics)    #与书本结果不一致原因：源码错误，欧氏距离计算时未取平方根
itemsim['Lady in the Water'][:2]

[(0.4, 'You, Me and Dupree'), (0.2857142857142857, 'The Night Listener')]

In [206]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings = prefs[user]
    scores={}
    totalSim={}
    for (item, rating) in userRatings.items():      #同时取出元组的两个值，再按任意顺序调用
        for (similarity, item2) in itemMatch[item]:
            if item2 in userRatings: continue
            scores.setdefault(item2,0)
            scores[item2]+= similarity*rating
            
            totalSim.setdefault(item2,0)
            totalSim[item2]+= similarity
    rankings = [(score/totalSim[item], item) for item, score in scores.items()]
    rankings.sort()
    rankings.reverse()
    return rankings

In [154]:
getRecommendedItems(critics, itemsim, 'Toby')

[(3.1826347305389224, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

## Movie Lens数据集
数据集[下载](https://grouplens.org/datasets/movielens/)

In [155]:
def loadMoviLens(path='data/movielens'):
    movies={}
    for line in open(path + '/u.item', encoding='latin-1'):
        (id,title) = line.split('|')[0:2]
        movies[id] = title
    
    prefs={}
    for line in open(path + '/u.data',encoding='latin-1'):
        (user, movieid, rating, ts)= line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    
    return prefs

In [156]:
prefs=loadMoviLens()

In [207]:
getRecommendations(prefs,'87')[:5]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) ')]

In [202]:
itemsim=calculateSimilarItems(prefs,n=50)

In [203]:
getRecommendedItems(prefs,itemsim,'87')[:5]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)')]

# 基于用户的推荐

In [208]:
def calculateSimilarUsers(prefs,n=10):
    result={}
    
    for item  in prefs:
        scores=topMatches(prefs,item,n=n,similarity=sim_pearson)
        result[item]=scores
    return result

In [211]:
usersim = calculateSimilarUsers(prefs,n=2)

# 音乐推荐系统
基于[Audioscrobbler](http://www.audioscrobbler.net/)